In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import json
import geopandas as gpd
from shapely.wkt import loads

In [28]:
# loading the main dataframe
main_df = pd.read_csv("20181024_d1_0830_0900_segmented_oneaxistrajectory_traffic.csv")
# loading the traffic light states
with open("20181024_d1_0830_0900_traffic_light_states.json") as f:
    traffic_info = json.load(f)
# loading the segments
segments_gdf = gpd.read_file("20181024_d1_0830_0900_segments.csv")
# Convert to GeoDataFrame
if "geometry" in segments_gdf.columns:
    # Convert 'geometry' to shapely objects if needed
    segments_gdf["geometry"] = segments_gdf["geometry"].apply(loads)
    # Create GeoDataFrame and set the CRS to WGS 84 (latitude/longitude)
    segments_gdf = gpd.GeoDataFrame(segments_gdf, geometry="geometry", crs="EPSG:4326")
else:
    raise ValueError("The DataFrame does not have a 'geometry' column.")

# Reproject to a projected CRS (replace EPSG code with appropriate UTM zone)
segments_gdf = segments_gdf.to_crs("EPSG:25832")  # Example for UTM Zone 32N

# Calculate lengths
segments_gdf["length"] = segments_gdf.geometry.length

# Calculate total length
total_length = segments_gdf["length"].sum()

# loading the traffic lights
traffic_lights_df = pd.read_csv("traffic_lights.csv")
# loading the traffic light states
with open("traffic_info_dict.pkl", "rb") as f:
    traffic_lights_dict_states = pickle.load(f)

# Normalize the one axis trajectory

In [ ]:
from CTM_class import CTMParameters, initialize_density, update_cell_status
from helper import veh_to_density
# Initialize CTM parameters
ctm_params = CTMParameters()
ctm_params.cell_length = 
# Initialize densities for each segment
segment_densities = {idx: initialize_density() for idx in segments_gdf.index}
from sklearn.preprocessing import normalize
a = (main_df["one_axis_trajectory"]-main_df["one_axis_trajectory"].min())
a = a/a.max()


0         0.224291
1         0.224831
2         0.225371
3         0.225911
4         0.226451
            ...   
672406    0.213225
672407    0.213765
672408    0.214305
672409    0.214575
672410    0.215115
Name: one_axis_trajectory, Length: 672411, dtype: float64

In [30]:
segments_gdf

,geometry,length
0,"LINESTRING (3540931.311 2953774.261, 3540895.9...",40.142563
1,"LINESTRING (3540895.934 2953793.231, 3540860.5...",40.142427
2,"LINESTRING (3540860.556 2953812.201, 3540825.1...",40.142291
3,"LINESTRING (3540825.179 2953831.171, 3540789.8...",40.142155
4,"LINESTRING (3540789.802 2953850.141, 3540754.4...",40.142018
5,"LINESTRING (3540754.425 2953869.111, 3540719.0...",40.141882
6,"LINESTRING (3540719.049 2953888.081, 3540683.6...",40.141746
7,"LINESTRING (3540683.672 2953907.051, 3540648.2...",40.141610
8,"LINESTRING (3540648.296 2953926.021, 3540612.9...",40.141473
9,"LINESTRING (3540612.919 2953944.991, 3540577.5...",40.141337
